In [1]:
using PyCall
np = pyimport("numpy")

PyObject <module 'numpy' from '/tigress/zequnl/conda_envs/ps/lib/python3.8/site-packages/numpy/__init__.py'>

In [2]:
freq = 143.0
int_freq = Int(freq)

143

In [3]:
using HDF5

flux_143 = h5read("/tigress/zequnl/radio/lagache_sed_massage/catalog_$(143.0).h5", "flux");

flux = h5read("/tigress/zequnl/radio/lagache_sed_massage/catalog_$(freq).h5", "flux")
θ = h5read("/tigress/zequnl/radio/lagache_sed_massage/catalog_$(freq).h5", "theta")
ϕ = h5read("/tigress/zequnl/radio/lagache_sed_massage/catalog_$(freq).h5", "phi");

In [4]:
using XGPaint
using Healpix
hp = pyimport("healpy")

using PyPlot
using Unitful, UnitfulAstro
import PhysicalConstants.CODATA2018: BoltzmannConstant, PlanckConstant, SpeedOfLightInVacuum

# CHANGE FOR DIFFERENT FREQUENCY ==========================
# print(KCMB_to_Jy_factor(freq * 1u"GHz"))

const TCMB = 2.725u"K"
xf(ν) = float(PlanckConstant) * ν / float(BoltzmannConstant) / TCMB
MJy_mul(ν) = 1.05e3 * expm1(xf(ν))^2 * exp(-xf(ν)) * (ν/100u"GHz")^(-4)
tSZ_mul(ν) = 1e6u"1/K" * TCMB * (xf(ν) * (exp(xf(ν))+1) / expm1(xf(ν)) - 4)

tSZ_mul (generic function with 1 method)

In [ ]:
MJy_mul(freq * 1u"GHz"), tSZ_mul(freq * 1u"GHz")

# Generate Radio Map

In [ ]:
nside = 4096
m_radio = Map{Float64,RingOrder}(nside)
# flux_cut = 7e-3
# cut_array = flux_143 .< flux_cut  # in Jy
# XGPaint.catalog2map!(m_radio, flux[cut_array], θ[cut_array], ϕ[cut_array])
XGPaint.catalog2map!(m_radio, flux, θ, ϕ)

MJy_factor = MJy_mul(freq * 1u"GHz")
m_radio .*= MJy_factor / 1e6;  # CONVERT TO muK, note that our maps are in Jy/sr

In [ ]:
using FITSIO

m_cib = readMapFromFITS("/tigress/zequnl/websky/cib_nu0$(int_freq).fits", 1, Float64);
m_cib.pixels .*= MJy_factor

tSZ_factor = tSZ_mul(freq * 1u"GHz")
m_tsz = readMapFromFITS("/tigress/zequnl/websky/tsz.fits", 1, Float64);
m_tsz .*= tSZ_factor;

In [ ]:
ENV["SCRATCH"] = "/tigress/zequnl/xgpaint/"
@time halo_pos, halo_mass = read_halo_catalog_hdf5(
    joinpath(ENV["SCRATCH"],"websky_halos-light.hdf5"));

In [ ]:
fwhm_143 = 2.2
beamed_radio = hp.sphtfunc.smoothing(m_radio.pixels, fwhm=fwhm_100);
beamed_cib = hp.sphtfunc.smoothing(m_cib.pixels, fwhm=fwhm_100);
beamed_tsz = hp.sphtfunc.smoothing(m_tsz.pixels, fwhm=fwhm_100);

In [ ]:
mass_cut_array = halo_mass .> 10^14.5
positions = halo_pos[:, mass_cut_array];

In [ ]:
norm(x) = sqrt(x[1]^2 + x[2]^2 + x[3]^2)
unitize(x) = x ./ norm(x)

In [ ]:
max_disc_in_arcmin = 6
max_disc = deg2rad(max_disc_in_arcmin / 60)

In [ ]:
using ProgressMeter

function get_discs(beamed_map::Array{T}, res, pos) where T
#     res = beamed_map.resolution
    radii = T[]
    disc_fluxes = T[]
    
    num_halos = size(pos, 2)
    @showprogress for i in 1:num_halos
        pixel_inds = hp.query_disc(nside, unitize(positions[:,i]), max_disc) .+ 1
        append!(disc_fluxes, beamed_map[pixel_inds])
        vec1 = unitize(positions[:,i]) #pix2vecRing(res, vec2pixRing(res, positions[:,i]...))
        vec2 = hp.pixelfunc.pix2vec(res.nside, pixel_inds .- 1) #[pix2angRing(res, pixind) for pixind in pixel_inds]
#         angdists = [hp.rotator.angdist(vec1, [vec2[1][i_discpix], vec2[2][i_discpix]])[1] for i_discpix in 1:length(pixel_inds) ]
        angdists = hp.rotator.angdist(vec1, vec2)
        append!(radii, angdists)
    end
    
    return radii, disc_fluxes
end

In [ ]:
using Interpolations
spline(x, y) = scale(interpolate(y, BSpline(Cubic(Line(OnGrid())))), x)
# spline_∂ₓ(f, x_grid) = spline(x_grid, [Interpolations.gradient(f, x)[1] for x in x_grid])


In [ ]:
# function get_profile(m, res)
#     r, f = get_discs(m, res, positions)
#     Δr = max_disc/30
#     radial_coord = Δr/100:Δr:max_disc
#     cumulative_fluxes = [sum(f[r .< r₀]) for r₀ in radial_coord];
#     profile = spline_∂ₓ(spline(radial_coord, cumulative_fluxes), radial_coord)
#     return profile
# end

In [ ]:
function get_profile(m, res)
    r, f = get_discs(m, res, positions)
    Δr = max_disc/30
    radial_coord = 0.0:Δr:max_disc
    cumulative_fluxes = [
        sum( f[((r₀-Δr) .< r) .& (r .< (r₀+Δr))] ) / (π * ((r₀+Δr)^2 - max(0.0, r₀-Δr)^2))
        for r₀ in radial_coord];
    cumulative_fluxes[1] = sum( f[r .< Δr] ) / (π * Δr^2) 
    
    profile = spline(radial_coord, cumulative_fluxes) #spline_∂ₓ(
        
#         , radial_coord)
    return profile
end

In [ ]:
mask = hp.read_map("/tigress/zequnl/websky/radiomask_143_7mJy.fits");

In [ ]:
profile_radio = get_profile(beamed_radio, m_radio.resolution)
profile_tsz = get_profile(beamed_tsz, m_radio.resolution)
profile_cib = get_profile(beamed_cib, m_radio.resolution);

# profile_radio = get_profile(m_radio.pixels .* mask, m_radio.resolution)
# profile_tsz = get_profile(m_tsz.pixels .* mask, m_radio.resolution)
# profile_cib = get_profile(m_cib.pixels .* mask, m_radio.resolution);

In [ ]:
# r, fr = get_discs(m_radio.pixels, m_radio.resolution, positions)
# r, fsz = get_discs(beamed_tsz, m_radio.resolution, positions)

In [ ]:
hp.cartview((m_radio.pixels), lonra=[-1,2], latra=[-1,1], max=3e2, title="radio")

In [ ]:
hp.cartview(mask, lonra=[-1,2], latra=[-1,1], title="mask")

In [ ]:
sum(m_radio.pixels .* mask), sum(m_radio.pixels)

In [ ]:
profile_radio = get_profile(m_radio.pixels .* mask, m_radio.resolution)
profile_tsz = get_profile(m_tsz.pixels .* mask, m_radio.resolution)
profile_cib = get_profile(m_cib.pixels .* mask, m_radio.resolution);

In [ ]:
Δr = max_disc/30
plotting_r = Δr/10:max_disc/1000:(max_disc-Δr)
r_arcmin = rad2deg.(plotting_r) .* 60

plot(r_arcmin, profile_radio.(plotting_r), "-", label="radio")
plot(r_arcmin, abs.(profile_tsz.(plotting_r)) ./ 10  , "-", label="|tSZ| / 10")
plot(r_arcmin, profile_cib.(plotting_r) ./ 10, "-", label="CIB / 10")
# plot(radial_coord, predict(approx, radial_coord), "-")
legend()
# yscale("log")
# title("unbeamed $(freq) GHz (masked)")
ylim(0.0, 1e13)
savefig("figures/beamed_143_masked.pdf")

In [ ]:
profile_radio = get_profile(m_radio.pixels, m_radio.resolution)
profile_tsz = get_profile(m_tsz.pixels, m_radio.resolution)
profile_cib = get_profile(m_cib.pixels, m_radio.resolution);

Δr = max_disc/30
plotting_r = Δr/10:max_disc/1000:(max_disc-Δr)
r_arcmin = rad2deg.(plotting_r) .* 60;

In [ ]:
plot(r_arcmin, profile_radio.(plotting_r), "-", label="radio")
plot(r_arcmin, abs.(profile_tsz.(plotting_r)) ./ 10  , "-", label="|tSZ| / 10")
plot(r_arcmin, profile_cib.(plotting_r) ./ 10, "-", label="CIB / 10")
# plot(radial_coord, predict(approx, radial_coord), "-")
legend()
# yscale("log")
# title("unbeamed $(freq) GHz (unmasked, no flux cut)")
ylim(0.0, 1e13)
tight_layout()
savefig("figures/beamed_143_unmasked_no_cut.pdf")

In [ ]:
# r_arcmin = rad2deg.(plotting_r) .* 60
# plot(r_arcmin, profile_radio.(plotting_r) ./ ( π .* ((plotting_r .+ Δr/50).^2 - (plotting_r .- Δr/50).^2)), "-", label="radio")
# plot(r_arcmin, abs.(profile_tsz.(plotting_r)) ./ ( π .* ((plotting_r .+ Δr/50).^2 - (plotting_r .- Δr/50).^2)), "-", label="|tSZ|")
# plot(r_arcmin, profile_cib.(plotting_r) ./ ( π .* ((plotting_r .+ Δr/50).^2 - (plotting_r .- Δr/50).^2)), "-", label="CIB")
# # plot(radial_coord, predict(approx, radial_coord), "-")
# legend()
# yscale("log")
# title(freq)
# # ylim(0.0, 0.5e20)

In [ ]:
rad2deg(sqrt(nside2pixarea(nside))) * 60

In [ ]:
360 * 60 / 3nside